### Compute TPSim matrix for all odors
8 x 8 matrices by electrode (significant in classification analyses)

In [1]:
from os.path import isfile, join, exists
from os import listdir, makedirs
import numpy as np
import matplotlib.pyplot as plt
from brainpipe.system import study
from scipy.stats import spearmanr, pearsonr
from itertools import product, combinations

In [2]:
def compute_tpsim(data0, data1, stat):
    """compute the Temporal Pattern Similarity (tpsim) for all combinations of
    single-trial power in 2 different conditions
    
    Parameters
    ----------
    data1, data2 : arrays
        Must be of shape (n_pts, n_trials)
    stat : string
        The stat correlation method to use. 'pearson' or 'spearman'
    Returns
    -------
    tpsim : value
        The mean TPSim value 
    """
    corr = pearsonr if stat == 'pearson' else spearmanr
    assert (
        data0.shape[0] == data1.shape[0]
    ), "Error: shape of trial1 and trial2 must have the same npts"
    
    n_trials0, n_trials1 = data0.shape[1],data1.shape[1]
    
    list_tpsim = np.zeros((n_trials0*n_trials1))
    for trial0, trial1 in product(range(n_trials0),range(n_trials1)):
        list_tpsim += corr(data0[:,trial0],data1[:,trial1])[0]
    return np.mean(list_tpsim)

In [12]:
###############################################################################
st = study('Olfacto')
# all meaning all electrodes present at E and R 
# not just the ones significant for Low High classif at E and R
path_files = join(st.path,'feature/TPSim_Enc_Ret_By_Odor_all/')
path2save = join(path_files, 'TPS_R_by_odors_matrix/')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

sessions = ['E','R']
su_codes = ['S0','S1','S2','S3','S4','S5']
freqs = ['0_theta', '1_alpha', '2_beta','3_gamma']

su_odor_groups = {'CHAF' : {'low':['1','2','4','5'],
                              'high':['3','8','7','9']},
                    'VACJ' : {'low':['11','14','12','10'],
                              'high':['15','17','16','13']},
                    'SEMC' : {'low':['7','10','11','12','13'],
                              'high':['5','8','9']},
                    'PIRJ' : {'low':['1','9','5'],
                              'high':['4','6','7','18']},
                    'LEFC' : {'low':['15','2','1','16'],
                              'high':['14','3','4','17']},
                    'FERJ' : {'low':['7','2','16','17'],
                              'high':['12','1','5','13']},}
rois_su_elecs = {
    'HC':{'FERJ':[[2],[8,9,19],[8,9],[]],
          'LEFC':[[2,8],[],[2,6],[]],
          'PIRJ':[[5,10,12],[9,10],[2,4,5,7,10,12],[]],
          'VACJ':[[],[1],[],[]]},
    'OFC':{'FERJ':[[],[],[38],[]],
           'LEFC':[[],[],[23],[]],
           'SEMC':[[],[],[41],[]],
           'PIRJ':[[],[],[22],[]]},
    'IFG':{'FERJ':[[41],[],[],[40]],
           'LEFC':[[28],[],[],[19]],
           'SEMC':[[27],[],[],[19]],
           'VACJ':[[14],[],[],[]]}}

for f,freq in enumerate(freqs):
    for roi in rois_su_elecs:
        for su in (sorted(rois_su_elecs[roi])):
            for elec in rois_su_elecs[roi][su][f]:
                print(freq,roi,su,elec)
                odors_list = su_odor_groups[su]['low']+su_odor_groups[su]['high']
                tpsim = np.zeros((len(odors_list),len(odors_list)))
                for i, od0 in enumerate(odors_list):
                    for j, od1 in enumerate(odors_list):
                        limit = len(su_odor_groups[su]['low'])
                        if (i < limit and j < limit) or (i >= limit and j >= limit):
                            mat0 = np.load(path_files+su+'_odor'+od0+'_E_pow'+freq+'.npz')
                            mat1 = np.load(path_files+su+'_odor'+od1+'_R_pow'+freq+'.npz')
                            data0 = mat0['xpow'][elec,:,:]
                            data1 = mat1['xpow'][elec,:,:]
                            tpsim[i,j] = compute_tpsim(data0, data1, 'spearman')
                        else:
                            tpsim[i,j] = 'Nan'
                
                #plot the matrix and save
                title = 'TPSim for patient '+su+' in '+roi+' ('+str(elec)+') in '+freq[2:]
                fig, ax = plt.subplots()
                plt.title(title)
                vmin, vmax = np.nanmin(tpsim), np.nanmax(tpsim)
                extr = abs(vmin) if abs(vmin)> vmax else vmax
                plt.imshow(tpsim,vmin=-extr,vmax=extr)
                odors = [int(i) for i in odors_list]
                ax.set_xticks(np.arange(len(odors_list)), minor=False)
                ax.set_yticks(np.arange(len(odors_list)), minor=False)
                ax.set_xticklabels(odors)
                ax.set_yticklabels(odors)
                cbar = plt.colorbar()
                cbar.set_label('TPSim',rotation=270)
#                 plt.show()
                filename = 'TPSim_'+su+'_'+roi+'_('+str(elec)+')_'+freq[2:]+'.png'
                plt.savefig(path2save+filename)
                plt.clf()
                plt.close()

-> Olfacto loaded
0_theta HC FERJ 2
0_theta HC LEFC 2
0_theta HC LEFC 8
0_theta HC PIRJ 5
0_theta HC PIRJ 10
0_theta HC PIRJ 12
0_theta IFG FERJ 41
0_theta IFG LEFC 28
0_theta IFG SEMC 27
0_theta IFG VACJ 14
1_alpha HC FERJ 8
1_alpha HC FERJ 9
1_alpha HC FERJ 19
1_alpha HC PIRJ 9
1_alpha HC PIRJ 10
1_alpha HC VACJ 1
2_beta HC FERJ 8
2_beta HC FERJ 9
2_beta HC LEFC 2
2_beta HC LEFC 6
2_beta HC PIRJ 2
2_beta HC PIRJ 4
2_beta HC PIRJ 5
2_beta HC PIRJ 7
2_beta HC PIRJ 10
2_beta HC PIRJ 12
2_beta OFC FERJ 38
2_beta OFC LEFC 23
2_beta OFC PIRJ 22
2_beta OFC SEMC 41
3_gamma IFG FERJ 40
3_gamma IFG LEFC 19
3_gamma IFG SEMC 19


In [ ]:
print(rois_su_elecs[roi][su][0])


In [ ]:
plt.imshow(np.random.random((50,50)));
plt.colorbar()
plt.show()